# Manual 03: Manejo de datos del DANE - Inseguridad alimentaria
## Universidad del Norte
### Economía Matemática

Este manual intenta mostrar un par de paquetes para una eficiente escritura de codigo y comprender la importancia de los procesos al manejar datos desde el **"seteo"** correcto de *directorios*, ejecucion de filtrados siguiendo la metodologia del DANE. Se definen funciones a partir de variable seleccionadas, como ademas  de la recategorizacion de valores de variables para mejor interpretacion.
 
Esta guia tiene como objetivo identifcar como para todo conjunto de datos se deben comprender los conceptos básicos como el uso de *keys*, uso de funciones de paquetes y/o correcta parametrizacion para poder **empalmar** tablas de una misma base de datos y conseguir armar modelos a conveniencia con los datos ya correctamente tratados.  

### Paso 1: Setear el directorio de trabajo
Para garantizar que los archivos se carguen correctamente, se recomienda establecer el **directorio de trabajo** donde se encuentran los archivos del DANE. Esto se puede hacer con el paquete `os`.

En este caso la ruta asignada debe ser donde esten los documentos que el paquete pandas va a leer.

In [1]:
import os
os.chdir("/Users/carlosandresyanesguerra/Documents/Ecomat")
# Recuerde siempre usar el / correcto  

print(os.listdir())
# La funcion listdir() sirve para verificar que los archivos que vamos a leer realmente existan 
# En el directorio, los nombres deben ser exactos de lo contrario pandas no lo reconocera. 

['.DS_Store', 'Filtered_masterdata.csv', 'Clase1.ipynb', 'Datosmodelo.csv', 'Opera1.html', 'EjercicioEVA', 'clase.html', 'Prom01.qmd', 'Opera1.ipynb', 'Opera1_files', '.git', 'Opera2.ipynb']


### Paso 2: Carga de Datos

Los archivos del DANE vienen en formato `CSV` y utilizan como delimitador (;). Se pueden cargar de la siguiente manera utilizando el paquete `pandas` que nos permite visualizar informacion de los llamados **dataframes**. 

In [2]:
import pandas as pd

tabla1 = pd.read_csv('ejercicioEVA/caracteristicas.CSV', delimiter=';')
tabla2 = pd.read_csv('ejercicioEVA/condicionesdevida.CSV', delimiter=';')
tabla3 = pd.read_csv('ejercicioEVA/datosdelavivienda.csv', delimiter=';')
tabla4 = pd.read_csv('ejercicioEVA/Educacion.CSV', delimiter=';')
tabla5 = pd.read_csv('ejercicioEVA/serviciosdelhogar.CSV', delimiter=';')

### Paso 3: Definicion de llaves e interseccion de datos. 

Entienda este proceso como el inicio del filtrado de posibles datos duplicados en su analisis, el **DANE** cuenta con varias herramientas para identificar el conjunto de datos de diferentes tablas en una misma base datos, para asegurarnos de tener una cantidad de datos que tenga sentido y sean relevantes, usaremos la variable `DIRECTORIO` como nuestra **llave** para el filtrado de datos.  

In [3]:
# No queremos cambiar la estructura de los archivos anteriormente definidos, los valores de los encuestados
# Es asignado a la variable DIRECTORIO recuerda esta es nuestra LLAVE

directorio1 = set(tabla1["DIRECTORIO"])
directorio2 = set(tabla2["DIRECTORIO"])
directorio3 = set(tabla3["DIRECTORIO"])
directorio4 = set(tabla4["DIRECTORIO"])
directorio5 = set(tabla5["DIRECTORIO"])

#Buscando las intersecciones en los sets

directorios_comunes = directorio1.intersection(directorio2,directorio3,directorio4,directorio5)
#asignamos los valores de la interseccion dentro de la variable directorios comunes

len(directorios_comunes)
#revisamos el largo de la interseccion para determinar que el numero de datos tenga sentido y sea relevante

86063

### Paso 4: Filtrado por valores comunes usando la funcion .isin() y eliminacion de valores duplicados. 

A continuación se hará uso de la funcion .isin() como una forma eficiente de registrar datos en un **dataframe**. En este caso, basado en una lista de valores dada por la variable `DIRECTORIO`. Sin embargo, tambien se puede usar para registrar datos coincidentes en otra tabla, luego se usa la funcion `drop.duplicates` para eliminar esos datos duplicados que compartian algo del mismo valor de la variable DIRECTORIO

In [4]:
#utilizamos el comando isin para buscar los datos de los directorios que se encuentren dentro de cada base de datos

tabla1_filtrada = tabla1[tabla1["DIRECTORIO"].isin(directorios_comunes)] 
tabla2_filtrada = tabla2[tabla2["DIRECTORIO"].isin(directorios_comunes)]
tabla3_filtrada = tabla3[tabla3["DIRECTORIO"].isin(directorios_comunes)]
tabla4_filtrada = tabla4[tabla4["DIRECTORIO"].isin(directorios_comunes)]
tabla5_filtrada = tabla5[tabla5["DIRECTORIO"].isin(directorios_comunes)]
#de esta forma se busca dentro de la lista de interseccion identificada anteriormente

# Eliminamos los valores duplicados en cada una de las tablas, con el fin de no tener datos repetidos

tabla1_filtrada = tabla1_filtrada.drop_duplicates(subset="DIRECTORIO")
tabla2_filtrada = tabla2_filtrada.drop_duplicates(subset="DIRECTORIO")
tabla3_filtrada = tabla3_filtrada.drop_duplicates(subset="DIRECTORIO")
tabla4_filtrada = tabla4_filtrada.drop_duplicates(subset="DIRECTORIO")
tabla5_filtrada = tabla5_filtrada.drop_duplicates(subset="DIRECTORIO")

### Paso 5: Sufijos antes de empalmar tablas

Los sufijos nos permiten evitar la perdida de columnas al hacer un empalme de tablas, para esto debemos usar la función `.add_suffix("")` y luego asegurar que la columna de la variable "llave" en este caso no tenga sufijo, de esta forma al realizar el **empalme** mayormente conocido como *merged*, nos quede todo emparejado con los valores en cada una de las tablas que establecimos anteriormente.  

In [5]:
tabla1_filtrada = tabla1_filtrada.add_suffix("_archivo1")
tabla2_filtrada = tabla2_filtrada.add_suffix("_archivo2")
tabla3_filtrada = tabla3_filtrada.add_suffix("_archivo3")
tabla4_filtrada = tabla4_filtrada.add_suffix("_archivo4")
tabla5_filtrada = tabla5_filtrada.add_suffix("_archivo5")
# Utilizamos el comando add_sufix para agregar una etiqueta

tabla1_filtrada["DIRECTORIO"] = tabla1_filtrada["DIRECTORIO_archivo1"]
tabla2_filtrada["DIRECTORIO"] = tabla2_filtrada["DIRECTORIO_archivo2"]
tabla3_filtrada["DIRECTORIO"] = tabla3_filtrada["DIRECTORIO_archivo3"]
tabla4_filtrada["DIRECTORIO"] = tabla4_filtrada["DIRECTORIO_archivo4"]
tabla5_filtrada["DIRECTORIO"] = tabla5_filtrada["DIRECTORIO_archivo5"]

### Paso 6: Guardar un nuevo documento

Para guardar todo el procedimiento y obtener la nueva *tabla merged* se usan diferentes funciones como: `.concat([...]`, `axis=1`, `join="inner"`, `.set_index` y `.reset_index()` 

#### Función pd.concat([...], axis=1, join="inner") 

- `pd.concat()` se usa para concatenar (unir) múltiples DataFrames. 
- `axis=1` Esto indica que los DataFrames se están combinando por columnas (de manera horizontal)
- `join="inner"` Realiza una intersección de los índices, es decir, solo se conservarán las filas cuyos DIRECTORIO estén presentes en todos los DataFrames.

#### Funciones .set_index() y .reset_index() 

Cada tabla antes de *concatenarse* cambia su índice a la columna DIRECTORIO, esto permite que la combinación se haga de manera precisa usando esta clave como referencia, esto convierte la columna DIRECTORIO en el índice del **DataFrame**. 

Después de la concatenación, el índice de la tabla resultante sigue siendo DIRECTORIO. Si queremos que DIRECTORIO vuelva a ser una columna normal en lugar del índice, usamos .reset_index().


In [6]:
masterdata = pd.concat([tabla1_filtrada.set_index("DIRECTORIO"),tabla2_filtrada.set_index("DIRECTORIO"),tabla3_filtrada.set_index("DIRECTORIO"),tabla4_filtrada.set_index("DIRECTORIO"),tabla5_filtrada.set_index("DIRECTORIO")],axis=1,join="inner").reset_index()

#El comando head nos permite ver si obtenimos el resultado esperado
masterdata.head()

,DIRECTORIO,DIRECTORIO_archivo1,SECUENCIA_ENCUESTA_archivo1,SECUENCIA_P_archivo1,ORDEN_archivo1,FEX_C_archivo1,P6016_archivo1,P1894_archivo1,P6020_archivo1,P6034_archivo1,...,P3174S1_archivo5,P3174S2_archivo5,P3174S3_archivo5,P3174S4_archivo5,P3174S5_archivo5,I_HOGAR_archivo5,I_UGASTO_archivo5,PERCAPITA_archivo5,I_OU_archivo5,CANT_PERSONAS_HOGAR_archivo5
0,7910114,7910114,1,1,1,282.370945,1,3,2,1,...,NaN,NaN,NaN,NaN,NaN,775000.0,775000.0,387500.0,0.0,2
1,7910115,7910115,1,1,1,3.054217,1,3,1,1,...,NaN,1.0,NaN,NaN,NaN,2200000.0,2200000.0,440000.0,0.0,5
2,7910119,7910119,1,1,1,31.288477,1,3,2,1,...,NaN,NaN,NaN,NaN,NaN,410000.0,410000.0,205000.0,0.0,2
3,7910120,7910120,1,1,1,73.577749,1,3,2,1,...,NaN,NaN,NaN,NaN,NaN,720000.0,720000.0,360000.0,0.0,2
4,7910121,7910121,1,1,1,48.549967,1,3,2,1,...,NaN,NaN,NaN,NaN,NaN,1892500.0,1892500.0,946250.0,0.0,2


### Paso 7: Lectura de archivos nuevos para saleccion de variables y lectura de documentos dentro de un directorio en la ruta principal

Primero verificaremos que la estructura de los datos sean iguales para saber que proceso vamos a seguir en el **empalme** y si se va requerir el uso de llaves para poder hacer el empalme como al principio. Para la lectura de archivos dentro de un directorio vamos a tomar las rutas bases de todos los archivos de los cuales vamos a seleccionar variables en una lista. 

## Estructura de archivo Atlantico

In [7]:
doc1 = r'/Users/carlosandresyanesguerra/Documents/Ecomat/EjercicioEVA/B data calidad de vida/Base de Calidad de vida Atlantico.dta'
doc11 = pd.read_stata(doc1)
doc11.head()

,DIRECTORIO,SECUENCIA_ENCUESTA,SECUENCIA_P,ORDEN,DEPARTAMENTO,MUNICIPIO,REGION,FEX_C,CANT_HOG_COMPLETOS,CANT_HOGARES_VIVIENDA,CLASE,P3516,X01,X02,X03,X04,X05,X06,X07,X08
0,7910520,1,1,1,8,1,1,496.065748,1,1,1,NaN,1,1,1,1,1,1,2,2
1,7910548,1,1,1,8,758,1,4.835424,1,1,2,NaN,1,2,1,1,1,1,1,2
2,7910734,1,1,1,8,1,1,516.738612,1,1,1,NaN,2,2,2,2,2,2,2,2
3,7910990,1,1,1,8,1,1,509.852073,1,1,1,NaN,2,2,2,2,2,2,2,2
4,7911030,1,1,1,8,573,1,20.597219,1,1,2,NaN,1,1,1,1,2,2,2,2


## Estructura archivo Bolivar

In [8]:
doc2 = r'/Users/carlosandresyanesguerra/Documents/Ecomat/EjercicioEVA/B data calidad de vida/Base de Calidad de vida Bolivar.dta'
doc22 = pd.read_stata(doc2)
doc22.head()

,DIRECTORIO,SECUENCIA_ENCUESTA,SECUENCIA_P,ORDEN,DEPARTAMENTO,MUNICIPIO,REGION,FEX_C,CANT_HOG_COMPLETOS,CANT_HOGARES_VIVIENDA,CLASE,P3516,X01,X02,X03,X04,X05,X06,X07,X08
0,7910186,1,1,1,13,1,1,221.608387,1,1,2,NaN,1,1,1,1,1,2,2,2
1,7910187,1,1,1,13,1,1,143.166971,1,1,2,NaN,2,2,2,2,2,2,2,2
2,7910280,1,1,1,13,1,1,549.758837,1,1,1,NaN,2,2,2,2,2,2,2,2
3,7910283,1,1,1,13,1,1,98.321580,1,1,2,NaN,1,1,1,1,1,2,2,2
4,7910303,1,1,1,13,1,1,399.300546,1,1,1,NaN,1,1,1,2,2,2,2,1


Como los datos cumplen una misma estructura no es necesario usar llaves para el empalme ya que los archivos `.dta` tienen la ventaja de que al reconocer las columnas identificadoras DIRECTORIO, SECUENCIA_P y SECUENCIA_ENCUESTA, los archivos de *tipo* `.dta` pueden crear indices secuenciales agregando el parámetro `ignore_index = TRUE` y los datos empalmaran correctamente, ya que este índice seguira la misma secuencia con todos los archivos al cumplir con la misma estructura de datos.

In [9]:
file_paths = [
    r'/Users/carlosandresyanesguerra/Documents/Ecomat/EjercicioEVA/B data calidad de vida/Base de Calidad de vida Atlantico.dta',
    r'/Users/carlosandresyanesguerra/Documents/Ecomat/EjercicioEVA/B data calidad de vida/Base de Calidad de vida Bolivar.dta',
    r'/Users/carlosandresyanesguerra/Documents/Ecomat/EjercicioEVA/B data calidad de vida/Base de Calidad de vida Cesar.dta',
    r'/Users/carlosandresyanesguerra/Documents/Ecomat/EjercicioEVA/B data calidad de vida/Base de Calidad de vida Cordoba.dta',
    r'/Users/carlosandresyanesguerra/Documents/Ecomat/EjercicioEVA/B data calidad de vida/Base de Calidad de vida La Guajira.dta',
    r'/Users/carlosandresyanesguerra/Documents/Ecomat/EjercicioEVA/B data calidad de vida/Base de Calidad de vida Magdalena.dta',
    r'/Users/carlosandresyanesguerra/Documents/Ecomat/EjercicioEVA/B data calidad de vida/Base de Calidad de vida Sucre.dta'
]
Datadep = []

# Haciendo una lectura de cada archivo con el append o agrupación de data dentro de la lista
for file_path in file_paths:
    df = pd.read_stata(file_path)
    Datadep.append(df)

# Realizamos finalmente la concatenación de los df
Datadepartamentos = pd.concat(Datadep, ignore_index=True)

# Revisando que la data tenga sentido
print(Datadepartamentos.head())


   DIRECTORIO  SECUENCIA_ENCUESTA  SECUENCIA_P  ORDEN  DEPARTAMENTO  \
0     7910520                   1            1      1             8   
1     7910548                   1            1      1             8   
2     7910734                   1            1      1             8   
3     7910990                   1            1      1             8   
4     7911030                   1            1      1             8   

   MUNICIPIO  REGION       FEX_C  CANT_HOG_COMPLETOS  CANT_HOGARES_VIVIENDA  \
0          1       1  496.065748                   1                      1   
1        758       1    4.835424                   1                      1   
2          1       1  516.738612                   1                      1   
3          1       1  509.852073                   1                      1   
4        573       1   20.597219                   1                      1   

   CLASE  P3516  X01  X02  X03  X04  X05  X06  X07  X08  
0      1    NaN    1    1    1    1    1

## Paso 8: Juntar el dataframe masterdata con el nuevo archivo de Datadepratamentos 

Aquí usaremos denuevo la parte de key `DIRECTORIO` y la funcion `.isin`, verificamos si hay residuales despues del *merged* y creamos un archivo `.csv` con la unidad maestra o "master" filtrada e integrada con la nueva información de las demás bases de datos.

In [10]:
#Filtrando nuestra primera base de datos basado en los directorios utilizados en el primer estudio

Filtered_masterdata = masterdata[masterdata["DIRECTORIO"].isin(Datadepartamentos["DIRECTORIO"])]
len(Filtered_masterdata)

#Verificando el numero de residuales despues del merge

if len(Filtered_masterdata)- len(Datadepartamentos) == 0:
    
    print ("Todos los datos fueron encontrados")
    
else:
    
    print ("No todos los datos se encontraron")

Filtered_masterdata.to_csv("Filtered_masterdata.csv", index=False)

Todos los datos fueron encontrados


###  Paso 9: Seleccion de variables y creacion del dataframe a trabajar.

A partir de ahora seleccionamos las variables, con las que buscamos crear la variable <span style="color:pink">Seguridad Alimentaria</span> en el caribe colombiano y buscamos que esta varaible este codificada de tal forma que en la que se pueda categorizar los datos y obtener interpretaciones y modelos interesantes, para esto los datos deben estar en un subconjunto o dataframe por aparte.

In [11]:
VARIABLES_MODELO = ["P5010",
               "P791","P3162","P3163","P5032","P6081","P6083",
               "P6088","P2061","P1070","P6160","P8586"]

DIRECTORIO = "DIRECTORIO"

# Filtrando las columnas que coinciden exactamente con los prefijos de exact_variables seguidos de un sufijo
DATOS_MODELO = [col for col in Filtered_masterdata.columns if any(col.startswith(var + '_') for var in VARIABLES_MODELO)]

# Filtrar el DataFrame para mantener solo esas columnas
DATOS_MODELO_DF = Filtered_masterdata.loc[:, Filtered_masterdata.columns.isin(DATOS_MODELO)]

# Mostrar el DataFrame filtrado
DATOS_MODELO_DF.head()

,P6081_archivo1,P6083_archivo1,P6088_archivo1,P2061_archivo1,P1070_archivo3,P6160_archivo4,P8586_archivo4,P5010_archivo5,P791_archivo5,P3162_archivo5,P3163_archivo5,P5032_archivo5
3,3,3,1.0,1.0,1,1,2,1,3,NaN,NaN,4
63,2,2,9.0,2.0,1,1,2,1,3,NaN,NaN,4
64,2,2,4.0,2.0,1,1,2,2,1,NaN,NaN,4
140,2,2,2.0,2.0,2,1,2,2,1,1.0,70000.0,1
143,3,1,NaN,2.0,1,1,2,2,1,1.0,41582.0,1


### Paso 11: Reasignar la llave Directorio al dataframe de las variables 

Ya hemos creado el dataframe objetivo con las variables de los archivos `.csv`, lo bueno de haber creado el archivo `filtered_masterdata` es que la columna *DIRECTORIO* esta organizado con las columnas de nuestras variables tomadas, así que ahora crearemos otro **dataframe** que integre estas variables para poder trabjar con las variables del nuevo conjuntos de datos que eran de los archivos `.dta.`

In [12]:
if DIRECTORIO in Filtered_masterdata.columns:
    DATOS_MODELO = [DIRECTORIO] + DATOS_MODELO
    
DATAF1 =Filtered_masterdata[DATOS_MODELO]

DATAF1.head()

DATAF1.to_csv("Datosmodelo.csv", index= False)

### Paso 12: Identificacion de variables del conjunto de datos .dta 

Ya con las variables de los archivos `.csv` estan asignados a un archivo **datosmodelo.csv** ahora tenemos que trabajar con las variables de "calidad de vida.dta" para poder crear nuestra variable de <span style="color:pink">inseguridad alimentaria</span>, ahora se toma lo que unicamente ocurre cuando los valores de las variables seleccionadas sean estrictamente igual a 1 y asignar el valor de 1 en la variasble. Si por algun motivo el valor es diferente de 1, habrá que asignar el valor 0, ya estamos cerca!

$$Variable_i = \left\{
\begin{array}{ll}
1 & \text{si cumple la característica} \\
0 & \text{de lo contrario}
\end{array}
\right.$$


In [13]:
modificar_columnas = ["X01","X02","X03","X04","X05","X06","X07","X08"]

for columnas in modificar_columnas:
    Datadepartamentos[columnas] = Datadepartamentos[columnas].apply(lambda x:1 if x==1 else 0)

In [14]:
Datadepartamentos.head(10)

,DIRECTORIO,SECUENCIA_ENCUESTA,SECUENCIA_P,ORDEN,DEPARTAMENTO,MUNICIPIO,REGION,FEX_C,CANT_HOG_COMPLETOS,CANT_HOGARES_VIVIENDA,CLASE,P3516,X01,X02,X03,X04,X05,X06,X07,X08
0,7910520,1,1,1,8,1,1,496.065748,1,1,1,NaN,1,1,1,1,1,1,0,0
1,7910548,1,1,1,8,758,1,4.835424,1,1,2,NaN,1,0,1,1,1,1,1,0
2,7910734,1,1,1,8,1,1,516.738612,1,1,1,NaN,0,0,0,0,0,0,0,0
3,7910990,1,1,1,8,1,1,509.852073,1,1,1,NaN,0,0,0,0,0,0,0,0
4,7911030,1,1,1,8,573,1,20.597219,1,1,2,NaN,1,1,1,1,0,0,0,0
5,7911031,1,1,1,8,573,1,24.094573,1,1,2,NaN,1,1,0,0,0,0,0,0
6,7911049,1,1,1,8,1,1,607.704351,1,1,1,NaN,0,0,0,1,1,0,0,0
7,7911050,1,1,1,8,1,1,497.369340,1,1,1,NaN,1,1,1,1,1,1,1,0
8,7911062,1,1,1,8,1,1,604.370204,1,1,1,NaN,0,1,0,0,0,0,0,0
9,7911072,1,1,1,8,296,1,547.080870,1,1,1,NaN,0,1,0,0,0,0,0,0


### Paso 13: Agregar la columna indice al dataframe 

Este dato nos interesa debido a que es la manera en la vamos a poder definir una **función** que nos permita crear la variable inseguridad alimentaria, esta columna lo que hara es sumar a partir de "X01" hasta "X08" los valores de 1 de cada fila y almacenandolo en la columna indice.

In [15]:
Datadepartamentos["INDICE"] = Datadepartamentos[modificar_columnas].sum(axis=1)
Datadepartamentos.head(10)

,DIRECTORIO,SECUENCIA_ENCUESTA,SECUENCIA_P,ORDEN,DEPARTAMENTO,MUNICIPIO,REGION,FEX_C,CANT_HOG_COMPLETOS,CANT_HOGARES_VIVIENDA,...,P3516,X01,X02,X03,X04,X05,X06,X07,X08,INDICE
0,7910520,1,1,1,8,1,1,496.065748,1,1,...,NaN,1,1,1,1,1,1,0,0,6
1,7910548,1,1,1,8,758,1,4.835424,1,1,...,NaN,1,0,1,1,1,1,1,0,6
2,7910734,1,1,1,8,1,1,516.738612,1,1,...,NaN,0,0,0,0,0,0,0,0,0
3,7910990,1,1,1,8,1,1,509.852073,1,1,...,NaN,0,0,0,0,0,0,0,0,0
4,7911030,1,1,1,8,573,1,20.597219,1,1,...,NaN,1,1,1,1,0,0,0,0,4
5,7911031,1,1,1,8,573,1,24.094573,1,1,...,NaN,1,1,0,0,0,0,0,0,2
6,7911049,1,1,1,8,1,1,607.704351,1,1,...,NaN,0,0,0,1,1,0,0,0,2
7,7911050,1,1,1,8,1,1,497.369340,1,1,...,NaN,1,1,1,1,1,1,1,0,7
8,7911062,1,1,1,8,1,1,604.370204,1,1,...,NaN,0,1,0,0,0,0,0,0,1
9,7911072,1,1,1,8,296,1,547.080870,1,1,...,NaN,0,1,0,0,0,0,0,0,1


### Paso 14: Definicion de funciÓn para Inseguridad alimentaria de manera categórica y codificada.

Como ya se tienen las variables definidas con el conjunto de datos y la columna **indice**, usaremos estos valores que suma la columna indice cuya valor minimo es 0 y valor maximo es cero para crear una columna que represente la variable inseguridad alimentaria y categorizemos los niveles de esa inseguridad alimentaria.

#### Definición codificada


In [16]:
def seg_alimentaria(value):
    if value >= 0 and value <=3:
        return 0
    elif value >= 4 and value <=6:
        return 1
    elif value >= 7 and value <=8:
        return 2
    else:
        return "NA"
    
Datadepartamentos["SN_SA"] = Datadepartamentos["INDICE"].apply(seg_alimentaria)

Datadepartamentos.head(15)  

,DIRECTORIO,SECUENCIA_ENCUESTA,SECUENCIA_P,ORDEN,DEPARTAMENTO,MUNICIPIO,REGION,FEX_C,CANT_HOG_COMPLETOS,CANT_HOGARES_VIVIENDA,...,X01,X02,X03,X04,X05,X06,X07,X08,INDICE,SN_SA
0,7910520,1,1,1,8,1,1,496.065748,1,1,...,1,1,1,1,1,1,0,0,6,1
1,7910548,1,1,1,8,758,1,4.835424,1,1,...,1,0,1,1,1,1,1,0,6,1
2,7910734,1,1,1,8,1,1,516.738612,1,1,...,0,0,0,0,0,0,0,0,0,0
3,7910990,1,1,1,8,1,1,509.852073,1,1,...,0,0,0,0,0,0,0,0,0,0
4,7911030,1,1,1,8,573,1,20.597219,1,1,...,1,1,1,1,0,0,0,0,4,1
5,7911031,1,1,1,8,573,1,24.094573,1,1,...,1,1,0,0,0,0,0,0,2,0
6,7911049,1,1,1,8,1,1,607.704351,1,1,...,0,0,0,1,1,0,0,0,2,0
7,7911050,1,1,1,8,1,1,497.369340,1,1,...,1,1,1,1,1,1,1,0,7,2
8,7911062,1,1,1,8,1,1,604.370204,1,1,...,0,1,0,0,0,0,0,0,1,0
9,7911072,1,1,1,8,296,1,547.080870,1,1,...,0,1,0,0,0,0,0,0,1,0


#### Definición categórica

In [17]:
def seg_alimentaria_categ(value):
    if value == 0:
        return "Leve"
    elif value == 1:
        return "Moderada"
    elif value == 2:
        return "Severa"
    else:
        return "NA"
    
Datadepartamentos["SA_SA"] = Datadepartamentos["SN_SA"].apply(seg_alimentaria_categ)

Datadepartamentos.head(10)

,DIRECTORIO,SECUENCIA_ENCUESTA,SECUENCIA_P,ORDEN,DEPARTAMENTO,MUNICIPIO,REGION,FEX_C,CANT_HOG_COMPLETOS,CANT_HOGARES_VIVIENDA,...,X02,X03,X04,X05,X06,X07,X08,INDICE,SN_SA,SA_SA
0,7910520,1,1,1,8,1,1,496.065748,1,1,...,1,1,1,1,1,0,0,6,1,Moderada
1,7910548,1,1,1,8,758,1,4.835424,1,1,...,0,1,1,1,1,1,0,6,1,Moderada
2,7910734,1,1,1,8,1,1,516.738612,1,1,...,0,0,0,0,0,0,0,0,0,Leve
3,7910990,1,1,1,8,1,1,509.852073,1,1,...,0,0,0,0,0,0,0,0,0,Leve
4,7911030,1,1,1,8,573,1,20.597219,1,1,...,1,1,1,0,0,0,0,4,1,Moderada
5,7911031,1,1,1,8,573,1,24.094573,1,1,...,1,0,0,0,0,0,0,2,0,Leve
6,7911049,1,1,1,8,1,1,607.704351,1,1,...,0,0,1,1,0,0,0,2,0,Leve
7,7911050,1,1,1,8,1,1,497.369340,1,1,...,1,1,1,1,1,1,0,7,2,Severa
8,7911062,1,1,1,8,1,1,604.370204,1,1,...,1,0,0,0,0,0,0,1,0,Leve
9,7911072,1,1,1,8,296,1,547.080870,1,1,...,1,0,0,0,0,0,0,1,0,Leve


--------------------

-------------

Ya tenemos toda la estructura de las bases de datos y hasta acá el trabajo duro que habia por hacer.

# Opcional 

### Paso 15: Interpretar los datos o crear modelos. 

Ya teniendo todos los componentes que se necesitaban para dar estructura. Ahora podemos graficar que porcentaje de población en el caribe Colombiano que esta con <span style="color:pink">Inseguridad Alimentaria</span>  *leve*, *moderada* y *severa*.

```python
pip install plotly 
```

In [18]:
import plotly.express as px
import pandas as pd

# Calcular los porcentajes de cada categoría en SA_SA
category_counts = Datadepartamentos["SA_SA"].value_counts(normalize=True) * 100

# Convertir a DataFrame para Plotly
df_plotly = pd.DataFrame({"Categoría": category_counts.index, "Porcentaje": category_counts.values})

# Crear la gráfica de barras con Plotly
fig = px.bar(df_plotly, x="Categoría", y="Porcentaje", 
             text="Porcentaje",  # Muestra los valores sobre las barras
             color="Categoría", 
             color_discrete_sequence=px.colors.sequential.Viridis, 
             title="Distribución Porcentual de Individuos en inseguridad")

# Mejorar el formato de la gráfica
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(xaxis_title="Categoría", 
                  yaxis_title="Porcentaje (%)", 
                  template="plotly_white")

# Mostrar la gráfica
fig.show()

------------------------

------------------

# TALLER - INTERMEDIO 
## Creemos un modelo de regresión logistica multinomial.

Vamos a crear una **base de datos** común para crear un modelo de entrenamiento y luego crear el modelo real, tendremos que limpiar un poco mas los datos para este objetivo.


### PUNTO 1. Seleccionar columnas de interés 

Necesitamos seleccionar dos culumnas de interes del dataframe Datadepartamentos, estas columnas son la llave y la variable que representa la inseguridad alimenticia codificada.

Complete el codigo.

In [ ]:
# Seleccionar las columnas de interés de Datadepartamentos
df_departamentos_subset = Datadepartamentos[['', '']]

# Mostrar las primeras filas del subset
df_departamentos_subset.head()

### PUNTO 2. Empalmar la tabla creada con el dataframe DF1 usando la llave correspondiente.

Debe seleccionar y ubicar una función de pandas que utilice una columna como **llave** para poder empalmar ambas tablas debe elegir la opción correcta y completa el código

```python
a. pd.isna
b. pd.read_csv
c. pd.melt
d. pd.merge
```

Ademas escriba la llave donde corresponde ;)

In [ ]:
# Unir los DataFrames usando la columna
DATAF2 = (DATAF1, df_departamentos_subset, on='', how='left')

# Mostrar las primeras filas del DataFrame unido para verificar
DATAF2.head(20)

### PUNTO 3. Rellenar valores NaN 

Resulta que al verificar los valores `NaN` de las variables tomadas y eso es un problema. Por ejemplo:

In [ ]:
# Contar el número de valores NaN en cada columna
nan_counts = DATAF2.isna().sum()

# Mostrar el número de NaN en cada columna
print(nan_counts)

Queriamos usar la variable "P3163_archivo5" que representa cuanto pagaron el mes pasado o la ultima vez por el servicio de gas natural pero tiene un monton de valores `NaN`, Vamos a solucionar esto con una funcion que agrupe las medianas de los valores SN_SA 1, 2 y 3 y rellene los valores NaN con esas medianas. 

Al código le hace falta especificar si debe hacer el combinado de valores por columnas o por filas, ademas necesita saber cual es la variable de inseguridad alimentaria para la cual calculara la mediana para los diferentes grupos de **inseguridad alimentaria** de la variaable "P3163_archivo 5". 

Completelo...

In [ ]:
import numpy as np

# Suponiendo que DATAF2 es su DataFrame
# Calcule la mediana de P3163_archivo5 para cada grupo de SN_SA
medianas = DATAF2.groupby('')['P3163_archivo5'].median()
print(medianas)

# Función para aplicar la mediana según el valor de SN_SA
def reemplazar_nan(row):
    if pd.isna(row['P3163_archivo5']):
        return medianas[row['']]
    else:
        return row['P3163_archivo5']

# Aplicar la función para reemplazar los NaN en P3163_archivo5
DATAF2['P3163_archivo5'] = DATAF2.apply(reemplazar_nan, axis=)

DATAF2.head(20)

# Eliminar la columna 'P3162_archivo5' del DataFrame ya que tambien tenia muchos valores NaN
DATAF2.drop(columns=['P3162_archivo5'], inplace=True)

#### Vamos a terminar de limpiar los valores NaN

In [ ]:
# Eliminar filas con valores nulos
df_cleaned = DATAF2.dropna()

# Mostrar las dimensiones del DataFrame original y del limpio para comparación
print("Original:", DATAF2.shape)
print("Limpio:", df_cleaned.shape)

### PUNTO 4 Definición de variables del modelo y creación del modelo

Vamos a definir la variable dependiente y las variables explicativas y vamos a crear el modelo.

a. La primera parte del codigo es simplemente definir las variables, dato importante la función `.drop` esta excluyendo y no incluyendo por lo tanto cuidado.

In [ ]:
# Definir la variable dependiente y las variables explicativas
y = df_cleaned['']
X = df_cleaned.drop(columns=[])  # Solo excluyendo 'SN_SA' y 'DIRECTORIO'

# Añadir una constante a las variables explicativas para el modelo logit
import statsmodels.api as sm
X = sm.add_constant(X)

b. Definamos el modelo, las dimensiones de entrenamiento queremos que sean del 80% por lo tanto del test deben ser del 20%. Por favor ajuste el modelo a las dimensiones dichas y verifique que las dimensiones te den como resultados los siguientes valores: (14465, 12) (3617, 12) (14465,) (3617,).

c. Luego simplemente ajuste el modelo logit multinomial con la variable independiente de entrenamiento y las variables dependientes de entrenamiento. 

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=, random_state=42)

# Mostrar las dimensiones de los conjuntos de datos
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Ajustar el modelo logit multinomial
model = sm.MNLogit()
result = model.fit()

# Resumen del modelo
print(result.summary())

## ¡Listo felicitaciones ya tiene un modelo logit multinomial!

Le dejo un script que le muestra los efectos marginales de las variable del modelo esta en usted como interpretarlos ;) (Recuerde que los marginales se interpretan como son las utilidades de los consumidores)

In [ ]:
# Debe Calcular efectos marginales promedio
mfx = result.get_margeff(at='overall')
print("Efectos marginales promedio:")
print(mfx.summary())

-------------------------------

# Agradecimientos
 Al Doctor Jose Luis Ramos y al Estudiante e investigador Junior del OCSA del departamento de Economía de la Universidad del Norte el joven Juan Camilo Pinedo